# Exploratory Data Analysis

This notebook is used for exploratory data analysis (EDA) of the dataset. It includes code for generating plots and summarizing key statistics.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

# Define the path to CSV files
csv_path = Path(r"C:\RnD\MedHack\archive\MedHack AI Hospital\csv")

# List all CSV files in the directory
csv_files = list(csv_path.glob('*.csv'))
print("Available CSV files:")
for file in csv_files:
    print(f"- {file.name}")

# Load all CSV files into a dictionary
data_dict = {}
for file in csv_files:
    try:
        data_dict[file.stem] = pd.read_csv(file)
        print(f"Successfully loaded: {file.name}")
    except Exception as e:
        print(f"Error loading {file.name}: {str(e)}")

# Display the first few rows of each dataset
for name, df in data_dict.items():
    print(f"\nFirst few rows of {name}:")
    display(df.head())

: 

In [ ]:
# Summary statistics
data.describe()

In [ ]:
# Visualizing missing values
plt.figure(figsize=(10, 6))
sns.heatmap(data.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values Heatmap')
plt.show()

In [ ]:
# Example of a plot
plt.figure(figsize=(12, 6))
sns.countplot(x='column_name', data=data)
plt.title('Count of Column Name')
plt.xticks(rotation=45)
plt.show()